https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
tf.__version__

'2.9.1'

In [4]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'data/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file(
    'petfinde_mini.zip',
    dataset_url,
    extract=True,
    cache_dir='data'
)

dataframe = pd.read_csv(csv_file)

1668792/1668792 [==============================] - 3s 2us/step


In [5]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [6]:
# In the original dataset, 'AdoptionSpeed' of 4 indicates a pet was not adopted
dataframe['target'] = np.where(dataframe['AdoptionSpeed'] == 4, 0, 1)

# Drop unused features
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

In [8]:
train, val, test = np.split(dataframe.sample(frac=1), [int(0.8 * len(dataframe)), int(0.9 * len(dataframe))])

In [9]:
print(f'{len(train)} training examples')
print(f'{len(val)} validation examples')
print(f'{len(test)} test examples')

9229 training examples
1154 validation examples
1154 test examples


In [12]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [13]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

C:\Users\luisc\AppData\Local\Temp\ipykernel_6032\1208008271.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}


In [15]:
[(train_features, label_batch)] = train_ds.take(1)
print(f'Every feature: {list(train_features.keys())}')
print(f'A batch o ages: {train_features["Age"]}')
print(f'A batch of targets: {label_batch}')

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']
A batch o ages: [[ 2]
 [ 1]
 [12]
 [36]
 [ 2]]
A batch of targets: [1 1 1 1 1]


In [16]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature
    normalizer = layers.Normalization(axis=None)

    # Prepare a Dataset that only yields the feature
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    return normalizer

In [17]:
photo_count_col = train_features['PhotoAmt']
layer = get_normalization_layer('PhotoAmt', train_ds)
layer(photo_count_col)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.5089997 ],
       [-0.8270252 ],
       [-0.19097419],
       [-0.5089997 ],
       [-0.8270252 ]], dtype=float32)>

In [18]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a layer that turns strings into integer indices
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)

    # Otherwise, create a layer that turns integer values into integer indices
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)

    # Prepare a 'tf.data.Dataset' that only yields the feature
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the set of possible values and assign them a fixed integer index
    index.adapt(feature_ds)

    # Encode the integer indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the layer, so you can use them, or incude them in the Keras Functional model later
    return lambda feature: encoder(index(feature))

In [19]:
test_type_col = train_features['Type']

test_type_layer = get_category_encoding_layer(
    name='Type',
    dataset=train_ds,
    dtype='string'
)

test_type_layer(test_type_col)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)>

In [21]:
test_age_col = train_features['Age']

test_age_layer = get_category_encoding_layer(
    name='Age',
    dataset=train_ds,
    dtype='int64',
    max_tokens=5
)

test_age_layer(test_age_col)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)>

In [23]:
batch_size = 256

train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

C:\Users\luisc\AppData\Local\Temp\ipykernel_6032\1208008271.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
C:\Users\luisc\AppData\Local\Temp\ipykernel_6032\1208008271.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
C:\Users\luisc\AppData\Local\Temp\ipykernel_6032\1208008271.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}


In [24]:
all_inputs = []
encoded_features = []

# Numerical features
for header in ['PhotoAmt', 'Fee']:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)

    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

In [25]:
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')

encoding_layer = get_category_encoding_layer(
    name='Age',
    dataset=train_ds,
    dtype='int64',
    max_tokens=5
)

encoded_age_col = encoding_layer(age_col)

all_inputs.append(age_col)
encoded_features.append(encoded_age_col)

In [26]:
categorical_cols = [
    'Type',
    'Color1',
    'Color2',
    'Gender',
    'MaturitySize',
    'FurLength',
    'Vaccinated',
    'Sterilized',
    'Health',
    'Breed1'
]

for header in categorical_cols:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')

    encoding_layer = get_category_encoding_layer(
        name=header,
        dataset=train_ds,
        dtype='string',
        max_tokens=5
    )

    encoded_categorical_col = encoding_layer(categorical_col)

    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

In [28]:
all_features = layers.concatenate(encoded_features)
x = layers.Dense(32, activation='relu')(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)

In [29]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [30]:
# Use 'rankdir=LR' to make the graph horizontal
tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [31]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


c:\Users\luisc\miniconda3\envs\structured_data_classifier\lib\site-packages\keras\engine\functional.py:566: UserWarning: Input dict contained keys ['target'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


37/37 [==============================] - 4s 23ms/step - loss: 0.6661 - accuracy: 0.4963 - val_loss: 0.5439 - val_accuracy: 0.7357
Epoch 2/10
37/37 [==============================] - 0s 11ms/step - loss: 0.5821 - accuracy: 0.6643 - val_loss: 0.5289 - val_accuracy: 0.7496
Epoch 3/10
37/37 [==============================] - 0s 11ms/step - loss: 0.5618 - accuracy: 0.6867 - val_loss: 0.5226 - val_accuracy: 0.7357
Epoch 4/10
37/37 [==============================] - 0s 11ms/step - loss: 0.5506 - accuracy: 0.6935 - val_loss: 0.5180 - val_accuracy: 0.7366
Epoch 5/10
37/37 [==============================] - 0s 11ms/step - loss: 0.5422 - accuracy: 0.7073 - val_loss: 0.5154 - val_accuracy: 0.7305
Epoch 6/10
37/37 [==============================] - 0s 11ms/step - loss: 0.5378 - accuracy: 0.7099 - val_loss: 0.5132 - val_accuracy: 0.7331
Epoch 7/10
37/37 [==============================] - 0s 11ms/step - loss: 0.5343 - accuracy: 0.7138 - val_loss: 0.5128 - val_accuracy: 0.7296
Epoch 8/10
37/37 [======

In [32]:
loss, accuracy = model.evaluate(test_ds)
print(f'Accuracy: {accuracy}')

5/5 [==============================] - 0s 8ms/step - loss: 0.4856 - accuracy: 0.7617
Accuracy: 0.7616984248161316


In [33]:
model.save('models/my_pet_classifier')
reloaded_model = tf.keras.models.load_model('models/my_pet_classifier')

INFO:tensorflow:Assets written to: models/my_pet_classifier\assets


INFO:tensorflow:Assets written to: models/my_pet_classifier\assets


In [38]:
sample = {
    'Type': 'Cat',
    'Age': 3,
    'Breed1': 'Tabby',
    'Gender': 'Male',
    'Color1': 'Black',
    'Color2': 'White',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    'Vaccinated': 'No',
    'Sterilized': 'No',
    'Health': 'Healthy',
    'Fee': 100,
    'PhotoAmt': 2
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(f'This particular pet had {100 * prob}% probability of getting adopted')

1/1 [==============================] - 0s 48ms/step
This particular pet had [76.06907]% probability of getting adopted
